In [1]:
import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [2]:
train = pd.read_json("data/training.json")

In [25]:
contexts = []
questions = []
answers_text = []
answers_start = []
is_impossible = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'] if len(q_a['answers']) > 0 else None)
            answers_text.append(q_a['answers'][0]['text'] if len(q_a['answers']) > 0 else None)
            is_impossible.append(q_a['is_impossible'])
            contexts.append(sub_para['context'])   
train_df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text, "target": is_impossible})

In [12]:
valid = pd.read_json("data/development.json")

In [26]:
contexts = []
questions = []
answers_text = []
answers_start = []
is_impossible = []
for i in range(valid.shape[0]):
    topic = valid.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'] if len(q_a['answers']) > 0 else None)
            answers_text.append(q_a['answers'][0]['text'] if len(q_a['answers']) > 0 else None)
            is_impossible.append(q_a['is_impossible'])
            contexts.append(sub_para['context'])   
valid_df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text, "target": is_impossible})

In [14]:
test = pd.read_json("data/test.json")

In [15]:
contexts = []
questions = []
for i in range(test.shape[0]):
    topic = test.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            contexts.append(sub_para['context'])   
test_df = pd.DataFrame({"context":contexts, "question": questions})

In [4]:
with open("data/dict_embeddings.pickle", "rb") as f:
    dict_emb = pickle.load(f)

In [27]:
def process_data(df):
    df['sentences'] = df['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    df['sent_emb'] = df['sentences'].apply(lambda x: [dict_emb[item][0] if item in dict_emb else np.zeros(4096) for item in x])
    df['quest_emb'] = df['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
    return df

In [28]:
train_df = process_data(train_df)

In [29]:
valid_df = process_data(valid_df)

In [30]:
test_df = process_data(test_df)

In [31]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [32]:
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(cosine(item,x["quest_emb"][0]))
    return li

In [33]:
np.warnings.filterwarnings('ignore')
train_df["cosine_sim"] = train_df.apply(cosine_sim, axis = 1)
valid_df["cosine_sim"] = valid_df.apply(cosine_sim, axis = 1)
test_df["cosine_sim"] = test_df.apply(cosine_sim, axis = 1)

In [34]:
def get_candidate(distances):
    return np.argmin(distances)

In [41]:
train_df = train_df[train_df["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

ValueError: malformed node or string: ['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.', "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.", "Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time.", 'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".']

In [61]:
len(train_df["sentences"][44])

5

In [62]:
def create_features(df):
    df_vect = pd.DataFrame()
    for k in range(len(df["cosine_sim"])):
        for i in range(len(df["cosine_sim"][k])):
            df_vect.loc[k, "column_cos_"+"%s"%i] = df["cosine_sim"][k][i]
            
    df_vect["target"] = df["target"]
    return df_vect

In [ ]:
train_vect = create_features(train_df)

In [ ]:
train_vect.head()